# 处理字词

In [ ]:
import glob
import string
import re
import copy
import math
import nltk  
import nltk.data 
import numpy as np

## train set

### 正则表达式部分

In [ ]:
# 得到所有文章的字符串
raw = ''

paths = glob.glob('./Training_Data/*.TXT')
for path in paths:
    raw += open(path).read()


def splitSentence(paragraph):  
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')  
    sentences = tokenizer.tokenize(paragraph)  
    return sentences  
  
if __name__ == '__main__': 
    # 删除括号和括号之间的内容
    raw = re.sub("\(.*?\)|\(|\)|\{.*?}|\{|\}|\[.*?]|\[|\]",'', raw)
    
    # 多个 *+\n 变成 .\n
    raw = re.sub("\*+",r'.', raw)
    
    # 多个.\n\n+ 变成 .\n
    raw = re.sub(r"\.\n\n+",r'.\n', raw)
    
    # 多个 \n\n+ 变成 .
    raw = re.sub(r"\n\n+",r'. ', raw)
    
    raw = raw.replace('\n',' ')
    
    #print(raw[:100],len(raw))
    sentences = splitSentence(raw)

### 特殊字符和标点符号部分

In [ ]:
# 移除含有特殊字符的句子
special_symbol = '@#$%^&*_+-=;:\"\'<>/~`'
for i in range(len(sentences)-1,-1,-1):
    for letter in special_symbol:
        if letter in sentences[i]:
            sentences.pop(i)

# 标点符号变成' '
for i in range(len(sentences)-1,-1,-1):
    for letter in '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~':
        sentences[i] = sentences[i].replace(letter,' ')

          
# 去除前后的空格
for i in range(len(sentences)-1,-1,-1):
    sentences[i] = sentences[i].strip()
    
# 去除前后的'.'
for i in range(len(sentences)-1,-1,-1):
    sentences[i] = sentences[i].strip('.')
    
# 去除前后的空格
for i in range(len(sentences)-1,-1,-1):
    sentences[i] = sentences[i].strip()

         
# 如果全部不全是数字或者字母，就删除
for i in range(len(sentences)-1,-1,-1):
    if re.sub(r" ",r'', sentences[i]).isalnum() == False:
        sentences.pop(i)


# 多个空格转一个空格
for i in range(len(sentences)-1,-1,-1):
    sentences[i] = re.sub(r" +",r' ', sentences[i])
    
# 去除前后的空格
for i in range(len(sentences)-1,-1,-1):
    sentences[i] = sentences[i].strip()


# 移除长度小于6的句子
for i in range(len(sentences)-1,-1,-1):
    if len(sentences[i].split(' ')) < 6:
        sentences.pop(i)
print('total sentences: ',len(sentences))

### save

In [ ]:
f=open('train-sentences.txt','w')
for sentence in sentences:
    f.write(sentence.lower()+"\n")
f.close()

## dev set for n-gram

### n-gram dev集准备，把答案替换入句子中

In [ ]:
dev = open('./development_set.txt').read()

dev = re.sub(r"\n\n\n",'#', dev)

dev_questions = dev.split('#')
print(len(dev_questions))
dev_questions = dev_questions[0:-1]
print(len(dev_questions))

write_out_list = []

for dev_question in dev_questions:
    
    question_num = dev_question[:dev_question.find(')')]
    
    question = dev_question[dev_question.find(' ')+1:dev_question.find('\n')]
    
    
    a_answer = dev_question[dev_question.find('a)')+3:dev_question.find('\n',dev_question.find('a)')+3)]
    b_answer = dev_question[dev_question.find('b)')+3:dev_question.find('\n',dev_question.find('b)')+3)]
    c_answer = dev_question[dev_question.find('c)')+3:dev_question.find('\n',dev_question.find('c)')+3)]
    d_answer = dev_question[dev_question.find('d)')+3:dev_question.find('\n',dev_question.find('d)')+3)]
    e_answer = dev_question[dev_question.find('e)')+3:]
    
    
    write_out_list.append(question_num)
    write_out_list.append(question.replace('_____',a_answer))
    write_out_list.append(question.replace('_____',b_answer))
    write_out_list.append(question.replace('_____',c_answer))
    write_out_list.append(question.replace('_____',d_answer))
    write_out_list.append(question.replace('_____',e_answer))
    

# 标点符号变成' '
for i in range(len(write_out_list)-1,-1,-1):
    for letter in string.punctuation:
        write_out_list[i] = write_out_list[i].replace(letter,' ')

# 多个' '变一个' '，去掉前后的' '
for i in range(len(write_out_list)):
    write_out_list[i] = re.sub(' +',' ',write_out_list[i])
    write_out_list[i] = write_out_list[i].strip() 
# 变成小写
for i in range(len(write_out_list)-1,-1,-1):
    write_out_list[i] = write_out_list[i].lower()

### save

In [ ]:
f=open('dev-sentences.txt','w')
for sentence in write_out_list:
    f.write(sentence+"\n")
f.close()